In [19]:
import cv2
from darkflow.net.build import TFNet

import matplotlib.pyplot as plt
from sys import exit

import numpy as np
import copy
%config inlineBackend.figure_format = 'svg'

In [17]:
# 학습
options = {
    'model' : 'cfg/lp_detect.cfg',
    'backup' : 'test/1/ckpt/',
    #'load' : 28979,
     #이전 학습 가중치를 이어서 학습(-1은 마지막 save를 불러옴)
    'load' : -1,
    #데이터셋의 전체 크기를 batch만큼 나눔
    'batch': 4,
    #학습의 횟수
    'epoch': 1,
    #gpu 사용 여부
    'gpu' : 0.7,
    'train' : True,
    # learning rate
    'lr' : 0.00001,
    #annotation의 위치
    'annotation': 'test/training/annotations/',
    #dataset의 위치
    'dataset' : 'test/training/',
    'labels': '../labels.txt'
}

#tensorflow에서 darkflow 사용
tfnet = TFNet(options)

tfnet.train()

Parsing cfg/lp_detect.cfg
Loading None ...
Finished in 0.0009922981262207031s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Init  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Init  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 76, 7

FileNotFoundError: [Errno 2] No such file or directory: 'test/1/ckpt/checkpoint'

In [21]:
# 실행
options = {
    "model" : 'cfg/lp_detect.cfg',
   # 'load' : 'bin/yolo.weights',
    'backup' : './test/1/ckpt/',
    'load' : -1,#43441,
    #confidence가 threshold보다 높을 경우 바운딩박스를 수용하겠다는 의미
    'threshold' : 0.1,
    'gpu' : 0.8,
    'labels': '../labels.txt'
}

tfnet = TFNet(options)
#tfnet.load_from_ckpt()

Parsing cfg/lp_detect.cfg
Loading None ...
Finished in 0.0009996891021728516s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Init  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Init  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 76, 7

FileNotFoundError: [Errno 2] No such file or directory: './test/1/ckpt/checkpoint'

In [ ]:
#이미지 파일을 color로 읽음
img = cv2.imread('test/training/images/licenseplates_2102.jpg', cv2.IMREAD_COLOR)
img1= img

#BGR 사진을 RGB 사진으로 변환
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#json형태로 결과 출력
results = tfnet.return_predict(img)
print((results))

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
#plot으로 원본 이미지 출력
fig = plt.figure(figsize=(19, 12)) 
plt.imshow(img1)
plt.show()

#이미지 3차원 행렬(y축, x축, 바로 색을 표현하는 BGR)
img.shape

#랜덤한 색으로 추출된 바운딩 박스와 라벨 표시
resultimg = copy.deepcopy(img)
colors = [tuple(255*np.random.rand(3)) for _ in range(1000)]
for color, result in zip(colors, results):
    t1 = (result['topleft']['x'], result['topleft']['y'])
    br = (result['bottomright']['x'], result['bottomright']['y'])
    label = result['label']
    confidence = result['confidence']
    
    print(confidence,t1,br)
    crop_img = img[t1[1]:br[1], t1[0]:br[0]]
    plt.imshow(crop_img)
    plt.show()

    resultimg = cv2.rectangle(resultimg, t1, br, color, 2)
    resultimg = cv2.putText(resultimg, label, t1, cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 2)

#plot으로 결과가 표시된 이미지 출력
fig = plt.figure(figsize=(19, 12)) 
plt.imshow(resultimg)
plt.show()


In [ ]:
index = 0

print((results[index]))
t1 = (results[index]['topleft']['x'], results[index]['topleft']['y'])
br = (results[index]['bottomright']['x'], results[index]['bottomright']['y'])
print(t1,br)
crop_img = img[t1[1]:br[1], t1[0]:br[0]]
plt.imshow(crop_img)
plt.show()

In [ ]:
cv2.imwrite("c:/project/data/result/result.jpg",crop_img)